In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
with open(filter_list_company_name_path) as f:
    filter_list_company_name = f.read().splitlines()
with open(filter_list_company_name_part_path) as f:
    filter_list_company_name_part = f.read().splitlines()

with open(filter_list_posting_region_path) as f:
    filter_list_posting_region = f.read().splitlines()
with open(filter_list_posting_name_part_path) as f:
    filter_list_posting_name_part = f.read().splitlines()

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

3735


### 1. 필터링 - 회사명에 부분적으로 포함되는 단어

In [4]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1

    print(filtering_word, nCnt)

학원 0
스터디 0
교육원 0
교육센터 0
평생교육원 0
필라테스 0
아카데미 0
병원 0
부동산 0
중개 0
건축사 0
의원 0
결혼 0
출판 0
손해사정 0
축구 0
스쿨 0
학교 0
산부인과 0
주얼리 0
부티크 0
복지관 0
스투디오 0
에스씨에이게임 0
렌트카 0
스타트업 0
의료센터 0
웨딩 0
에이치알 0
테라로사 0
성형외과 0
변호사 0
화장품 0
대한상공회의소 0
안과 0
복지재단 0
투어 0
체형교정 0
사단법인 0
투자자문 0
노무법인 0
수영장 0
자산운용 0
폴리텍 0
울산 0
특허법인 0
위너TFK 0
에듀플렉스 0
법률사무소 0
천안 0
충북 0
법무법인 0
도시락 0
리조트 0
요양원 0
산후조리원 0
사회복지 0
청소년 0
인재개발원 0
컨벤션 0
인베스트먼트 0
교육개발원 0
장애인 0
사무소 0


### 2. 필터링 - 공고명에 부분적으로 포함되는 단어

In [5]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 0
매장관리 0
고객 0
사무보조 0
마케팅 0
마케터 0
경영지원 0
NodeJs 0
Nestjs 0
부동산 0
영업사원 0
자재관리 0
회계 0
재무 0
총무 0
영상의학실 0
유지보수 0
고객응대 0
텔레마케터 0
아웃바운드 0
국비지원 0
화장품 0
디자이너 0
인플루언서 0
MD 0
AMD 0
온라인 0
네일리스트 0
웹디자인 0
기술영업 0
회로설계 0
강사 0
경리 0
사무행정 0
보험설계사 0
유튜브 0
유투브 0
국비무료 0
국비지원 0
수행기사 0
행정보조 0
구축사업 0
골프장 0
영업지원 0
Node.js 0
node.js 0
방송장비 0
물류팀 0
물류관리 0
창고관리 0
정보보안 0
품질경영 0
코스메틱 0
자동차튜닝 0
시각디자인 0
악세사리 0
업무제휴 0
무료 0
영상제작 0
생산관리 0
인사팀장 0
자동차영업 0
생산팀 0
창업센터 0
가맹점 0
웹퍼블리셔 0
품질관리 0
임원비서 0
뷰티 0
해외영업 0
중국어 0
새벽배송 0
센터관리 0
무역업무 0
비서 0
해외구매 0
광고영업 0
기획영업 0
영업기획부 0
유학 0
세무법인 0
세무회계 0
광주 0
법률사무소 0
교육재단 0


In [6]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

### 3. 전체현황 - 회사명에 부분적으로 단어 포함

In [7]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                    nCnt += 1

    print(filtering_word, nCnt)

학원 0
스터디 0
교육원 0
교육센터 0
평생교육원 0
필라테스 0
아카데미 0
병원 0
부동산 0
중개 0
건축사 0
의원 0
결혼 0
출판 0
손해사정 0
축구 0
스쿨 0
학교 0
산부인과 0
주얼리 0
부티크 0
복지관 0
스투디오 0
에스씨에이게임 0
렌트카 0
스타트업 0
의료센터 0
웨딩 0
에이치알 0
테라로사 0
성형외과 0
변호사 0
화장품 0
대한상공회의소 0
안과 0
복지재단 0
투어 0
체형교정 0
사단법인 0
투자자문 0
노무법인 0
수영장 0
자산운용 0
폴리텍 0
울산 0
특허법인 0
위너TFK 0
에듀플렉스 0
법률사무소 0
천안 0
충북 0
법무법인 0
도시락 0
리조트 0
요양원 0
산후조리원 0
사회복지 0
청소년 0
인재개발원 0
컨벤션 0
인베스트먼트 0
교육개발원 0
장애인 0
사무소 2


### 4. 하나씩 확인 - 회사명에 부분적으로 단어 포함

In [8]:
filtering_word = "변호사"
ans_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['company']:
            ans_list.append(company_title)
print(filtering_word)
print("검출개수: " + str(len(ans_list)))
ans_list = list(set(ans_list))
print("중복제거: " + str(len(ans_list)))
print("----------------------")
for ans in ans_list:
    print(ans)

변호사
검출개수: 0
중복제거: 0
----------------------


### 5. 전체현황 - 공고명에 부분적으로 단어 포함

In [9]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 0
매장관리 0
고객 0
사무보조 0
마케팅 0
마케터 0
경영지원 0
NodeJs 0
Nestjs 0
부동산 0
영업사원 0
자재관리 0
회계 0
재무 0
총무 0
영상의학실 0
유지보수 0
고객응대 0
텔레마케터 0
아웃바운드 0
국비지원 0
화장품 0
디자이너 0
인플루언서 0
MD 0
AMD 0
온라인 0
네일리스트 0
웹디자인 0
기술영업 0
회로설계 0
강사 0
경리 0
사무행정 0
보험설계사 0
유튜브 0
유투브 0
국비무료 0
국비지원 0
수행기사 0
행정보조 0
구축사업 0
골프장 0
영업지원 0
Node.js 0
node.js 0
방송장비 0
물류팀 0
물류관리 0
창고관리 0
정보보안 0
품질경영 0
코스메틱 0
자동차튜닝 0
시각디자인 0
악세사리 0
업무제휴 0
무료 0
영상제작 0
생산관리 0
인사팀장 0
자동차영업 0
생산팀 0
창업센터 0
가맹점 0
웹퍼블리셔 0
품질관리 0
임원비서 0
뷰티 0
해외영업 0
중국어 0
새벽배송 0
센터관리 0
무역업무 0
비서 0
해외구매 0
광고영업 0
기획영업 0
영업기획부 0
유학 0
세무법인 0
세무회계 0
광주 0
법률사무소 0
교육재단 0


### 6. 일정 기한 지난 공고 삭제

In [8]:
import datetime
import json

# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print("회사 개수: ", len(job_group))

nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print("공고 개수: ", nCnt)
print("--------------")

회사 개수:  3468
공고 개수:  7867
--------------


In [9]:
gap_day = 45
removed_cnt = 0

for company_idx, company_title in enumerate(job_group):
    n_cnt = 0
    tmp_list = []

    for job_list in job_group[company_title]:
        input_date = job_list['input_date']
        date_input = datetime.date(int(input_date[0:4]), int(input_date[5:7]), int(input_date[8:10]))

        today = datetime.date.today().isoformat()
        date_today = datetime.date(int(today[0:4]), int(today[5:7]), int(today[8:10]))
        
        if (date_today - date_input).days < gap_day:
            tmp_list.append(job_list)
            tmp_list[n_cnt]['title_idx'] = n_cnt
            n_cnt += 1
        else:
            removed_cnt += 1
            print(job_list['company'], job_list['title'])
    job_group[company_title] = tmp_list

# 공고가 없는 회사명 제거    
del_temp = []
for company_idx, company_title in enumerate(job_group):
    if len(job_group[company_title]) == 0:
        del_temp.append(company_title)
for company in del_temp:
    del(job_group[company])
    
print('----------------')
print(removed_cnt)

로민 알고리즘 개발 엔지니어
로민 시니어 개발자 채용
한국축산데이터 정부과제담당자 모집 (신입/경력)
스파크엑스 [긴급] AI 실증사업 사업계획서 작성(프리랜서)
뉴빌리티 Operation Manager
케이티 2023년 R&D분야 경력/박사 채용(3차)
풍산시스템 자동화 설비 PC제어/머신비전 신입/경력직 채용
이노메트리 [2차전지 장비업] 딥러닝 및 컴퓨터 비전 알고리즘 개발자 모집
파인디지털 [차량 상황인지 개발] 부문 채용 (전문연구요원 가능)
인스케이프 (주)인스케이프 2023년 상반기 신입 채용 (SW 비전)
인스케이프 (주)인스케이프 2023년 상반기 신입/경력 채용 (SW 비전/제어)
인스케이프 (주)인스케이프 2023년 상반기 경력 채용 (연구소)
현대자동차 Robotics Engineer [LiDAR(PointCloud) Perception]
쿠팡 [쿠팡] 영업 사원 채용 (산업무관)
쿠팡 [쿠팡] 로켓그로스 / 제트배송 AM (Account Management) 상시채
쿠팡 [쿠팡] 로켓그로스 / 제트배송 AM (Account Management) 상시 채용
쿠팡 [쿠팡] 포토그래퍼/Photographer(계약직)
루트로닉 간편지원 광학 연구원 ( 외국계, 중견, 석사우대 )
루트로닉 간편지원 Hardware/Firmware Engineer (제어 하드웨어 펌웨어)
루트로닉 간편지원 의료기기 임상교육 (신입/경력)
디노마드 2023 디노마드 경영관리 파트장 채용
루켄테크놀러지스 [LUKEN] S/W 연구원 정규직 채용 (용인/분당/베트남)
투비소프트 [(주)투비소프트] 자동화설비 설계팀 경력 채용
투비소프트 [(주)투비소프트] 자동화설비 제어팀 경력 채용
투비소프트 [(주)투비소프트] 자동화설비 제어팀 팀장 채용
유일에너테크 2023 유일에너테크(주) 제어기술본부 신입/경력 모집
두플 [2023년] 제조 빅데이터 수집분석 솔루션개발자 (Java 고급 2명)
티에스에이 [TSA] 구매자재팀 정규직 신입 채용
티에스에이 [TSA] 시험그룹 신규

In [10]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

### 크롤링했던 날짜 분포 확인

In [3]:
import datetime
import json
from collections import defaultdict

date_dict = defaultdict(int)

# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print("회사 개수: ", len(job_group))

nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
        date_dict[j['input_date']] += 1


회사 개수:  3468


In [2]:
sorted(date_dict.items())

[('2023-04-03', 236),
 ('2023-04-05', 177),
 ('2023-04-06', 229),
 ('2023-04-08', 226),
 ('2023-04-09', 8),
 ('2023-04-10', 147),
 ('2023-04-11', 81),
 ('2023-04-14', 512),
 ('2023-04-18', 224),
 ('2023-04-19', 288),
 ('2023-04-23', 283),
 ('2023-04-24', 142),
 ('2023-04-25', 171),
 ('2023-04-27', 329),
 ('2023-04-29', 137),
 ('2023-04-30', 6),
 ('2023-05-01', 20),
 ('2023-05-02', 132),
 ('2023-05-04', 262),
 ('2023-05-05', 30),
 ('2023-05-07', 18),
 ('2023-05-08', 97),
 ('2023-05-10', 254),
 ('2023-05-11', 91),
 ('2023-05-12', 214),
 ('2023-05-13', 53),
 ('2023-05-14', 4),
 ('2023-05-15', 138),
 ('2023-05-16', 1984),
 ('2023-05-20', 968),
 ('2023-05-21', 406)]